# Quantization for LLM Optimization

This notebook demonstrates how to apply quantization techniques to further optimize our distilled student model. Quantization reduces the precision of model weights, enabling significant model size reduction with minimal impact on accuracy.

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time

# Add the src directory to the path
sys.path.append('..')

In [ ]:
# Import our modules
from src.models.distilbert import load_distilbert_model, get_device
from src.models.knowledge_distillation import create_student_model
from src.models.quantization import apply_dynamic_quantization, convert_to_fp16, export_to_onnx, QuantizedModelWrapper
from src.data.dataset import load_and_prepare_data, prepare_batch_for_model
from src.utils.metrics import measure_performance, save_metrics, print_metrics

In [ ]:
# Configuration
MODEL_NAME = "distilbert-base-uncased"
NUM_LABELS = 2  # Binary classification

# Dataset configuration
DATASET_NAME = "glue"
DATASET_CONFIG = "sst2"  # Stanford Sentiment Treebank
BATCH_SIZE = 16
MAX_LENGTH = 128

# Paths
OUTPUT_DIR = Path("../outputs")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
STUDENT_DIR = OUTPUT_DIR / "distilled_model"
QUANTIZED_DIR = OUTPUT_DIR / "quantized_model"
QUANTIZED_DIR.mkdir(exist_ok=True, parents=True)

# Device
DEVICE = get_device()
print(f"Using device: {DEVICE}")

## 1. Load Models and Data

Load the baseline teacher model and our distilled student model.

In [ ]:
# Load baseline metrics for comparison
print("Loading baseline metrics...")
try:
    baseline_metrics = torch.load(OUTPUT_DIR / "baseline_metrics.pt")
    student_metrics = torch.load(OUTPUT_DIR / "student_metrics.pt")
    print("Loaded metrics successfully.")
except FileNotFoundError:
    print("Warning: Metrics files not found. Run baseline_measurement.ipynb and knowledge_distillation.ipynb first.")

In [ ]:
# Load teacher model (for comparison)
print(f"Loading teacher model {MODEL_NAME}...")
teacher_model, tokenizer = load_distilbert_model(MODEL_NAME, NUM_LABELS)

# Load dataset
print("Loading dataset...")
tokenizer, train_dataloader, eval_dataloader = load_and_prepare_data(
    tokenizer, 
    dataset_name=DATASET_NAME, 
    dataset_config=DATASET_CONFIG,
    batch_size=BATCH_SIZE, 
    max_length=MAX_LENGTH
)

In [ ]:
# Create student model with the same architecture as used in knowledge distillation
print("Creating student model...")
student_model = create_student_model(
    teacher_model=teacher_model,
    num_layers=2,  # Using 2 layers as was done in knowledge distillation
    num_labels=NUM_LABELS
)

# Load student model weights if available
if STUDENT_DIR.exists():
    print(f"Loading student model weights from {STUDENT_DIR}...")
    # Load student model from saved files
    from transformers import DistilBertForSequenceClassification
    student_model = DistilBertForSequenceClassification.from_pretrained(STUDENT_DIR)
else:
    print("Warning: Student model directory not found. Run knowledge_distillation.ipynb first.")

## 2. Quantization Techniques

Apply different quantization techniques to the student model.

### 2.1 FP16 Quantization (Half Precision)

Convert floating-point numbers from 32-bit to 16-bit precision.

In [ ]:
# Apply FP16 quantization
print("Applying FP16 quantization...")
fp16_model, fp16_size_before, fp16_size_after = convert_to_fp16(student_model)

# Calculate size reduction
fp16_reduction = (fp16_size_before - fp16_size_after) / fp16_size_before * 100
print(f"FP16 quantization results:")
print(f"  Size before: {fp16_size_before:.2f} MB")
print(f"  Size after: {fp16_size_after:.2f} MB")
print(f"  Reduction: {fp16_reduction:.2f}%")

### 2.2 INT8 Dynamic Quantization

Apply dynamic quantization to convert weights to 8-bit integers.

In [ ]:
# Apply dynamic quantization
print("Applying dynamic INT8 quantization...")
try:
    int8_model, int8_size_before, int8_size_after = apply_dynamic_quantization(student_model, torch.qint8)
    
    # Calculate size reduction
    int8_reduction = (int8_size_before - int8_size_after) / int8_size_before * 100
    print(f"INT8 quantization results:")
    print(f"  Size before: {int8_size_before:.2f} MB")
    print(f"  Size after: {int8_size_after:.2f} MB")
    print(f"  Reduction: {int8_reduction:.2f}%")
    
    # Wrap the quantized model
    int8_model = QuantizedModelWrapper(int8_model)
except Exception as e:
    print(f"INT8 quantization failed: {e}")
    int8_model = None

## 3. Evaluate Quantized Models

Measure performance of the quantized models.

In [ ]:
# Define function to prepare batch
def prepare_batch(batch, device):
    return prepare_batch_for_model(batch, device)

In [ ]:
# Evaluate FP16 model
print("Evaluating FP16 quantized model...")
fp16_metrics = measure_performance(fp16_model, eval_dataloader, DEVICE, prepare_batch)

In [ ]:
# Evaluate INT8 model (if available)
if int8_model is not None:
    print("Evaluating INT8 quantized model...")
    try:
        int8_metrics = measure_performance(int8_model, eval_dataloader, 'cpu', prepare_batch)  # Use CPU for INT8
    except Exception as e:
        print(f"INT8 evaluation failed: {e}")
        int8_metrics = None

In [ ]:
# Save the metrics
save_metrics(fp16_metrics, file_path=OUTPUT_DIR / "fp16_metrics.pt")
if int8_metrics is not None:
    save_metrics(int8_metrics, file_path=OUTPUT_DIR / "int8_metrics.pt")

## 4. Compare Performance Across All Models

Compare the performance of the original, distilled, and quantized models.

In [ ]:
# Print metrics for all models
print("\n===== Teacher Model (Baseline) =====")
print_metrics(
    baseline_metrics, 
    model_name=MODEL_NAME, 
    dataset_info=f"Text Classification - {DATASET_NAME}/{DATASET_CONFIG}"
)

print("\n===== Student Model (Distilled) =====")
print_metrics(
    student_metrics, 
    model_name=f"Distilled {MODEL_NAME} (2 layers)", 
    dataset_info=f"Text Classification - {DATASET_NAME}/{DATASET_CONFIG}"
)

print("\n===== FP16 Model (Quantized) =====")
print_metrics(
    fp16_metrics, 
    model_name=f"FP16 Quantized Student", 
    dataset_info=f"Text Classification - {DATASET_NAME}/{DATASET_CONFIG}"
)

if int8_metrics is not None:
    print("\n===== INT8 Model (Quantized) =====")
    print_metrics(
        int8_metrics, 
        model_name=f"INT8 Quantized Student", 
        dataset_info=f"Text Classification - {DATASET_NAME}/{DATASET_CONFIG}"
    )

In [ ]:
# Calculate key metrics for comparison with baseline
models = ['Teacher', 'Student', 'FP16']
metrics_list = [baseline_metrics, student_metrics, fp16_metrics]

if int8_metrics is not None:
    models.append('INT8')
    metrics_list.append(int8_metrics)

# Extract metrics
sizes = [m['model_size_mb'] for m in metrics_list]
latencies = [m['avg_latency_ms'] for m in metrics_list]
accuracies = [m['accuracy'] * 100 for m in metrics_list]
params = [m['num_parameters'] / 1e6 for m in metrics_list]

# Calculate reductions
size_reductions = [(1 - size/sizes[0]) * 100 for size in sizes]
speed_improvements = [latencies[0]/latency for latency in latencies]
accuracy_retentions = [acc/accuracies[0] for acc in accuracies]

In [ ]:
# Create comparison table
import pandas as pd

comparison_df = pd.DataFrame({
    'Model': models,
    'Size (MB)': [f"{size:.2f}" for size in sizes],
    'Size Reduction (%)': [f"{reduction:.2f}" for reduction in size_reductions],
    'Accuracy (%)': [f"{acc:.2f}" for acc in accuracies],
    'Accuracy Retention (%)': [f"{retention*100:.2f}" for retention in accuracy_retentions],
    'Latency (ms)': [f"{latency:.2f}" for latency in latencies],
    'Speed Improvement (x)': [f"{improvement:.2f}" for improvement in speed_improvements],
    'Parameters (M)': [f"{param:.2f}" for param in params]
})

comparison_df

In [ ]:
# Create bar chart comparisons
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# Size comparison
axs[0, 0].bar(models, sizes)
axs[0, 0].set_title('Model Size (MB)')
axs[0, 0].set_ylabel('Size (MB)')
axs[0, 0].grid(axis='y')
for i, v in enumerate(sizes):
    axs[0, 0].text(i, v + 2, f"{v:.2f}", ha='center')

# Latency comparison
axs[0, 1].bar(models, latencies)
axs[0, 1].set_title('Inference Latency (ms)')
axs[0, 1].set_ylabel('Latency (ms)')
axs[0, 1].grid(axis='y')
for i, v in enumerate(latencies):
    axs[0, 1].text(i, v + 2, f"{v:.2f}", ha='center')

# Accuracy comparison
axs[1, 0].bar(models, accuracies)
axs[1, 0].set_title('Accuracy (%)')
axs[1, 0].set_ylabel('Accuracy (%)')
axs[1, 0].grid(axis='y')
for i, v in enumerate(accuracies):
    axs[1, 0].text(i, v + 1, f"{v:.2f}%", ha='center')

# Size reduction comparison
axs[1, 1].bar(models, size_reductions)
axs[1, 1].set_title('Size Reduction (%)')
axs[1, 1].set_ylabel('Reduction (%)')
axs[1, 1].grid(axis='y')
axs[1, 1].axhline(y=50, color='r', linestyle='--', label='50% Target')
axs[1, 1].legend()
for i, v in enumerate(size_reductions):
    axs[1, 1].text(i, v + 1, f"{v:.2f}%", ha='center')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'model_comparison.png')
plt.show()

## 5. Save the Best Quantized Model

Save the best performing quantized model for deployment.

In [ ]:
# Save FP16 model (typically the best balance of size and performance)
fp16_save_path = QUANTIZED_DIR / "fp16"
fp16_save_path.mkdir(exist_ok=True, parents=True)

# Save the model and tokenizer
fp16_model.save_pretrained(fp16_save_path)
tokenizer.save_pretrained(fp16_save_path)

print(f"FP16 quantized model saved to {fp16_save_path}")

In [ ]:
# Export to ONNX for deployment (optional)
try:
    print("Exporting to ONNX format...")
    onnx_path = QUANTIZED_DIR / "model.onnx"
    onnx_file, onnx_size = export_to_onnx(fp16_model, tokenizer, onnx_path)
    print(f"ONNX model exported to {onnx_file} (Size: {onnx_size:.2f} MB)")
except Exception as e:
    print(f"ONNX export failed: {e}")

## 6. Summary and Conclusion

Review the optimization results and overall performance.

In [ ]:
# Get the best model's metrics
best_metrics = fp16_metrics if fp16_metrics['accuracy'] >= (int8_metrics['accuracy'] if int8_metrics else 0) else int8_metrics
best_model_name = "FP16" if best_metrics is fp16_metrics else "INT8"

# Calculate final metrics
final_size_reduction = (1 - best_metrics['model_size_mb'] / baseline_metrics['model_size_mb']) * 100
final_speed_improvement = baseline_metrics['avg_latency_seconds'] / best_metrics['avg_latency_seconds']
final_accuracy_retention = best_metrics['accuracy'] / baseline_metrics['accuracy'] * 100

print("===== LLM Optimization Summary =====")
print(f"\nStarting point (Teacher model):")
print(f"  - Size: {baseline_metrics['model_size_mb']:.2f} MB")
print(f"  - Parameters: {baseline_metrics['num_parameters']:,}")
print(f"  - Accuracy: {baseline_metrics['accuracy']*100:.2f}%")
print(f"  - Inference time: {baseline_metrics['avg_latency_ms']:.2f} ms")

print(f"\nFinal optimized model (Distilled + {best_model_name} Quantization):")
print(f"  - Size: {best_metrics['model_size_mb']:.2f} MB")
print(f"  - Parameters: {best_metrics['num_parameters']:,}")
print(f"  - Accuracy: {best_metrics['accuracy']*100:.2f}%")
print(f"  - Inference time: {best_metrics['avg_latency_ms']:.2f} ms")

print(f"\nOptimization results:")
print(f"  - Size reduction: {final_size_reduction:.2f}%")
print(f"  - Speed improvement: {final_speed_improvement:.2f}x")
print(f"  - Accuracy retention: {final_accuracy_retention:.2f}%")

print("\nOptimization targets:")
print(f"  - Size target (50% reduction): {'✅ Success' if final_size_reduction >= 50 else '❌ Not reached'}")
print(f"  - Accuracy target (90% retention): {'✅ Success' if final_accuracy_retention >= 90 else '❌ Not reached'}")
print(f"  - Overall result: {'✅ All targets achieved!' if final_size_reduction >= 50 and final_accuracy_retention >= 90 else '❌ Some targets not reached'}")

print("\nConclusion:")
if final_size_reduction >= 50 and final_accuracy_retention >= 90:
    print("The combination of knowledge distillation and quantization successfully optimized the model")
    print(f"for edge deployment, achieving both our size reduction and accuracy retention targets.")
elif final_size_reduction >= 50:
    print("While we achieved our size reduction target, accuracy was impacted more than expected.")
    print("Consider fine-tuning the quantized model or exploring other optimization approaches.")
elif final_accuracy_retention >= 90:
    print("While accuracy remained high, we didn't quite reach our size reduction target.")
    print("Consider additional optimization techniques like pruning or further reducing the model architecture.")
else:
    print("Neither target was fully achieved. Consider revisiting the optimization approach")
    print("with different hyperparameters or techniques.")